# บทที่ 7: สร้างแอปพลิเคชันแชท
## เริ่มต้นใช้งาน OpenAI API

สมุดบันทึกนี้ดัดแปลงมาจาก [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) ซึ่งมีสมุดบันทึกสำหรับเข้าถึงบริการ [Azure OpenAI](notebook-azure-openai.ipynb)

OpenAI API สำหรับภาษา Python สามารถใช้งานร่วมกับโมเดลของ Azure OpenAI ได้เช่นกัน โดยมีการปรับแต่งเล็กน้อย ดูรายละเอียดเกี่ยวกับความแตกต่างเพิ่มเติมได้ที่นี่: [วิธีสลับใช้งานระหว่าง OpenAI และ Azure OpenAI endpoints ด้วย Python](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# ภาพรวม  
"โมเดลภาษาใหญ่คือฟังก์ชันที่แปลงข้อความหนึ่งไปเป็นข้อความอีกข้อความหนึ่ง เมื่อได้รับข้อความอินพุต โมเดลภาษาใหญ่จะพยายามทำนายข้อความถัดไปที่จะตามมา" (1) โน้ตบุ๊ก "เริ่มต้นอย่างรวดเร็ว" นี้จะพาผู้ใช้ไปรู้จักกับแนวคิดหลักของ LLM ข้อกำหนดแพ็กเกจสำคัญสำหรับการเริ่มต้นใช้งาน AML แนะนำเบื้องต้นเกี่ยวกับการออกแบบพรอมต์ และตัวอย่างสั้น ๆ ของการใช้งานในรูปแบบต่าง ๆ


## สารบัญ  

[ภาพรวม](../../../../07-building-chat-applications/python)  
[วิธีใช้งาน OpenAI Service](../../../../07-building-chat-applications/python)  
[1. การสร้าง OpenAI Service ของคุณ](../../../../07-building-chat-applications/python)  
[2. การติดตั้ง](../../../../07-building-chat-applications/python)    
[3. ข้อมูลรับรอง](../../../../07-building-chat-applications/python)  

[กรณีการใช้งาน](../../../../07-building-chat-applications/python)    
[1. สรุปข้อความ](../../../../07-building-chat-applications/python)  
[2. จัดประเภทข้อความ](../../../../07-building-chat-applications/python)  
[3. สร้างชื่อสินค้าขึ้นมาใหม่](../../../../07-building-chat-applications/python)  
[4. ปรับแต่งตัวจัดประเภท](../../../../07-building-chat-applications/python)  

[แหล่งอ้างอิง](../../../../07-building-chat-applications/python)


### สร้างพรอมต์แรกของคุณ  
แบบฝึกหัดสั้น ๆ นี้จะเป็นการแนะนำพื้นฐานสำหรับการส่งพรอมต์ไปยังโมเดลของ OpenAI เพื่อทำงานง่าย ๆ อย่าง "สรุปเนื้อหา"

**ขั้นตอน**:  
1. ติดตั้งไลบรารี OpenAI ในสภาพแวดล้อม python ของคุณ  
2. โหลดไลบรารีช่วยเหลือมาตรฐานและตั้งค่าข้อมูลรับรองความปลอดภัยของ OpenAI สำหรับบริการ OpenAI ที่คุณสร้างไว้  
3. เลือกโมเดลสำหรับงานของคุณ  
4. สร้างพรอมต์ง่าย ๆ สำหรับโมเดล  
5. ส่งคำขอของคุณไปยัง API ของโมเดล!


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. การเลือกโมเดลที่เหมาะสม  
โมเดล GPT-3.5-turbo หรือ GPT-4 สามารถเข้าใจและสร้างข้อความภาษาธรรมชาติได้


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. การออกแบบพรอมต์  

"ความมหัศจรรย์ของโมเดลภาษาใหญ่คือ เมื่อถูกฝึกให้ลดข้อผิดพลาดในการทำนายจากข้อความจำนวนมหาศาล โมเดลจะได้เรียนรู้แนวคิดที่เป็นประโยชน์ต่อการทำนายเหล่านี้ ตัวอย่างเช่น โมเดลจะได้เรียนรู้แนวคิดอย่างเช่น"(1):

* การสะกดคำ
* หลักไวยากรณ์
* การเขียนใหม่ให้ความหมายเหมือนเดิม
* การตอบคำถาม
* การสนทนา
* การเขียนได้หลายภาษา
* การเขียนโค้ด
* ฯลฯ

#### วิธีควบคุมโมเดลภาษาใหญ่  
"ในบรรดาข้อมูลนำเข้าทั้งหมดของโมเดลภาษาใหญ่ ข้อความพรอมต์มีอิทธิพลมากที่สุด"(1)

โมเดลภาษาใหญ่สามารถถูกกระตุ้นให้สร้างผลลัพธ์ได้หลายวิธี เช่น:

Instruction: บอกโมเดลว่าคุณต้องการอะไร
Completion: กระตุ้นให้โมเดลเขียนต่อจากจุดเริ่มต้นที่คุณต้องการ
Demonstration: แสดงให้โมเดลเห็นว่าคุณต้องการอะไร โดยใช้:
ตัวอย่างไม่กี่ตัวในพรอมต์
ตัวอย่างจำนวนมากในชุดข้อมูลฝึกแบบ fine-tuning



#### มีแนวทางพื้นฐาน 3 ข้อในการสร้างพรอมต์:

**แสดงและบอก**. ทำให้ชัดเจนว่าคุณต้องการอะไร ไม่ว่าจะผ่านคำสั่ง ตัวอย่าง หรือทั้งสองอย่างผสมกัน ถ้าคุณต้องการให้โมเดลจัดอันดับรายการตามตัวอักษร หรือจัดหมวดหมู่ย่อหน้าตามอารมณ์ ให้แสดงให้เห็นว่านี่คือสิ่งที่คุณต้องการ

**ให้ข้อมูลที่มีคุณภาพ**. ถ้าคุณพยายามสร้างตัวจำแนกประเภท หรืออยากให้โมเดลทำตามรูปแบบที่ต้องการ ต้องแน่ใจว่ามีตัวอย่างเพียงพอ อย่าลืมตรวจทานตัวอย่างของคุณด้วย — โมเดลมักจะฉลาดพอที่จะมองข้ามข้อผิดพลาดการสะกดคำพื้นฐานและตอบกลับได้ แต่บางครั้งอาจคิดว่าคุณตั้งใจและอาจส่งผลต่อคำตอบ

**ตรวจสอบการตั้งค่า**. การตั้งค่า temperature และ top_p จะควบคุมว่าโมเดลจะตอบแบบแน่นอนแค่ไหน ถ้าคุณต้องการคำตอบที่มีเพียงคำตอบเดียวที่ถูกต้อง ควรตั้งค่าให้น้อยลง แต่ถ้าต้องการคำตอบที่หลากหลายมากขึ้น อาจตั้งค่าสูงขึ้น ข้อผิดพลาดที่พบบ่อยที่สุดคือคิดว่าการตั้งค่านี้เป็นตัวควบคุม "ความฉลาด" หรือ "ความคิดสร้างสรรค์"


ที่มา: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## สรุปข้อความ  
#### ความท้าทาย  
สรุปข้อความโดยการเพิ่ม 'tl;dr:' ต่อท้ายข้อความ ดูว่าระบบสามารถเข้าใจและทำงานหลายอย่างได้โดยไม่ต้องมีคำสั่งเพิ่มเติม คุณสามารถลองใช้ prompt ที่อธิบายละเอียดกว่านี้แทน tl;dr เพื่อปรับพฤติกรรมของโมเดลและปรับแต่งการสรุปที่คุณต้องการได้(3)  

งานวิจัยล่าสุดแสดงให้เห็นว่าการฝึกโมเดลด้วยข้อมูลข้อความขนาดใหญ่ก่อน แล้วจึงปรับแต่งกับงานเฉพาะ ช่วยให้ได้ผลลัพธ์ที่ดีขึ้นในหลายงานและเกณฑ์วัดของ NLP แม้สถาปัตยกรรมจะไม่ได้ผูกกับงานใดงานหนึ่ง แต่ก็ยังต้องใช้ชุดข้อมูลสำหรับปรับแต่งที่มีตัวอย่างนับพันหรือนับหมื่น ในทางตรงข้าม มนุษย์มักจะทำงานภาษาใหม่ ๆ ได้จากตัวอย่างเพียงไม่กี่ตัวอย่างหรือแค่คำอธิบายง่าย ๆ ซึ่งระบบ NLP ปัจจุบันยังทำได้ไม่ดีนัก ที่นี่เราจะแสดงให้เห็นว่าการขยายขนาดโมเดลภาษา ช่วยเพิ่มประสิทธิภาพแบบไม่ผูกกับงานและแบบ few-shot ได้มากขึ้น บางครั้งก็สามารถแข่งขันกับวิธีการปรับแต่งที่ดีที่สุดในอดีตได้เลย



Tl;dr


# แบบฝึกหัดสำหรับหลายกรณีการใช้งาน  
1. สรุปข้อความ  
2. จัดประเภทข้อความ  
3. สร้างชื่อสินค้ารูปแบบใหม่


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## จัดหมวดหมู่ข้อความ  
#### ความท้าทาย  
จัดหมวดหมู่รายการต่าง ๆ ให้อยู่ในหมวดหมู่ที่กำหนดไว้ในขณะที่ทำการประมวลผล ในตัวอย่างต่อไปนี้ เราจะระบุทั้งหมวดหมู่และข้อความที่ต้องการจัดหมวดหมู่ไว้ในพรอมต์ (*playground_reference)

คำถามจากลูกค้า: สวัสดีค่ะ ปุ่มบนคีย์บอร์ดแล็ปท็อปของฉันหักไปหนึ่งปุ่มเมื่อไม่นานมานี้ และฉันต้องการเปลี่ยนใหม่:

หมวดหมู่ที่จัดไว้:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## สร้างชื่อสินค้ารูปแบบใหม่
#### โจทย์
สร้างชื่อสินค้าจากคำตัวอย่างที่ให้มา ในคำสั่งนี้จะมีข้อมูลเกี่ยวกับสินค้าที่เราต้องการตั้งชื่อให้ พร้อมตัวอย่างที่คล้ายกันเพื่อแสดงรูปแบบที่ต้องการ นอกจากนี้ยังตั้งค่า temperature สูงเพื่อให้ได้คำตอบที่หลากหลายและสร้างสรรค์มากขึ้น

รายละเอียดสินค้า: เครื่องทำมิลค์เชคสำหรับใช้ในบ้าน
คำตัวอย่าง: รวดเร็ว, สุขภาพดี, กะทัดรัด
ชื่อสินค้า: HomeShaker, Fit Shaker, QuickShake, Shake Maker

รายละเอียดสินค้า: รองเท้าที่สามารถปรับขนาดให้พอดีกับเท้าได้ทุกขนาด
คำตัวอย่าง: ปรับเปลี่ยนได้, พอดี, omni-fit


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# แหล่งข้อมูลอ้างอิง  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [ตัวอย่างจาก OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [แนวทางปฏิบัติที่ดีที่สุดสำหรับการปรับแต่ง GPT-3 เพื่อจัดประเภทข้อความ](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# สำหรับความช่วยเหลือเพิ่มเติม  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# ผู้ร่วมเขียน
* Louis Li



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารฉบับนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามอย่างเต็มที่เพื่อความถูกต้อง แต่โปรดทราบว่าการแปลโดยอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่มีความสำคัญ แนะนำให้ใช้บริการแปลโดยนักแปลมืออาชีพ ทางเราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่คลาดเคลื่อนซึ่งเกิดจากการใช้การแปลนี้
